In [1]:
import os
import fitz
from paddleocr import PaddleOCR
import torch

In [2]:
filename = os.path.join("pdf", "联通手机营业厅使用手册.pdf")
doc = fitz.open(filename)

In [4]:
page_count = doc.page_count
img_count = 0
img_dir = "img"
os.makedirs(img_dir, exist_ok=True)
for page_idx in range(page_count):
    # page = doc.load_page(page_idx)
    # text = page.get_text()
    # print(text)
    img_list = doc.get_page_images(page_idx)
    print(f"page[{page_idx}]: image num={len(img_list)}")
    for img in img_list:
        pix = fitz.Pixmap(doc, img[0])
        if pix.n - pix.alpha >= 4:
            pix = fitz.Pixmap(fitz.csRGB, pix)
        pix.save(os.path.join(img_dir, f"{img_count}.png"))
        img_count += 1

page[0]: image num=0
page[1]: image num=0
page[2]: image num=1
page[3]: image num=2
page[4]: image num=2
page[5]: image num=2
page[6]: image num=2
page[7]: image num=2
page[8]: image num=2
page[9]: image num=2
page[10]: image num=2
page[11]: image num=2
page[12]: image num=3
page[13]: image num=0


In [ ]:
def pdf_ocr_txt(filepath, dir_path="tmp_files"):
    full_dir_path = os.path.join(os.path.dirname(filepath), dir_path)
    if not os.path.exists(full_dir_path):
        os.makedirs(full_dir_path)
    ocr = PaddleOCR(use_angle_cls=True, lang="ch", use_gpu=torch.cuda.is_available(), show_log=False)
    doc = fitz.open(filepath)
    txt_file_path = os.path.join(full_dir_path, f"{os.path.split(filepath)[-1]}.txt")
    img_name = os.path.join(full_dir_path, 'tmp.png')
    with open(txt_file_path, 'w', encoding='utf-8') as fout:
        for i in range(doc.page_count):
            page = doc[i]
            text = page.get_text("")
            fout.write(text)
            fout.write("\n")

            img_list = page.get_images()
            for img in img_list:
                pix = fitz.Pixmap(doc, img[0])
                if pix.n - pix.alpha >= 4:
                    pix = fitz.Pixmap(fitz.csRGB, pix)
                pix.save(img_name)

                result = ocr.ocr(img_name)
                ocr_result = [i[1][0] for line in result for i in line]
                fout.write("\n".join(ocr_result))
    if os.path.exists(img_name):
        os.remove(img_name)
    return txt_file_path